In [3]:
import time
from getpass import getpass
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common import NoSuchElementException, ElementClickInterceptedException
import pathlib
import os
from os.path import join
import pandas as pd
import random
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [13]:
all_schools =pd.read_csv('final_allschools_df_temporary.csv')

In [20]:
all_schools.columns

Index(['Unnamed: 0', 'name', 'address', 'link', 'details', 'n_students',
       'neighborhood', 'type'],
      dtype='object')

In [26]:
all_schools.drop('Unnamed: 0', axis=1, inplace=True)

In [29]:
all_schools.type.value_counts()

Basic School     5624
Middle School    1575
Name: type, dtype: int64

In [30]:
middle_schools = all_schools[all_schools.type == 'Middle School']

In [33]:
primary_schools = all_schools[all_schools.type == 'Basic School']

In [35]:
middle_schools.details.unique()

array(['OverigeVoortgezet onderwijs535 leerlingen',
       'OverigeVoortgezet onderwijs491 leerlingen',
       'Prot. christelijkVoortgezet onderwijs1019 leerlingen', ...,
       'Algemeen bijzonderVoortgezet speciaal onderwijs185 leerlingen',
       'OpenbaarVoortgezet onderwijs1244 leerlingen',
       'OpenbaarVoortgezet speciaal onderwijs385 leerlingen'],
      dtype=object)

In [64]:
primary_schools.details.unique()

array(['Prot. christelijkBasisonderwijs84 leerlingen',
       'OpenbaarBasisonderwijs44 leerlingen',
       'SamenwerkingBasisonderwijs116 leerlingenDalton', ...,
       'KatholiekBasisonderwijs424 leerlingen',
       'Prot. christelijkSpeciaal basisonderwijs150 leerlingen',
       'Prot. christelijkBasisonderwijs317 leerlingenIPC'], dtype=object)

In [65]:
primary_schools['details'] = primary_schools['details'].apply(clean_details)

In [66]:
primary_schools


,name,address,link,details,n_students,neighborhood,type
1575,Kindcentrum De Wegwijzer,Harm Tiesingstraat 74,/basisscholen/2e-exloërmond/25159/kindcentrum-...,Prot. christelijk,84.0,2e Exloërmond,Basic School
1576,Openbare Basisschool De Westhoek,Schoolstraat 27,/basisscholen/2e-exloërmond/7595/openbare-basi...,Openbaar Basisonderwijs44,44.0,2e Exloërmond,Basic School
1577,Basisschool De Tweeklank,Albardastraat 35,/basisscholen/aadorp/11785/basisschool-de-twee...,Samenwerking Basisonderwijs116,116.0,Aadorp,Basic School
1578,JHR Willem Versluijsschool,Prelaatweg 22,/basisscholen/aagtekerke/25963/jhr-willem-vers...,Prot. christelijk,191.0,Aagtekerke,Basic School
1579,Christelijke Basisschool de Schutse,Paardelandsdrift 9,/basisscholen/aalden/9661/christelijke-basissc...,Prot. christelijk,62.0,Aalden,Basic School
...,...,...,...,...,...,...,...
7194,De Ichthus,Zwarteweg 74,/basisscholen/zwolle/9451/de-ichthus/,Prot. christelijk,317.0,Zwolle,Basic School
7195,De Zuidster,Munterkamp 71,/basisscholen/zwolle/9452/de-zuidster/,Prot. christelijk,87.0,Zwolle,Basic School
7196,PC Basisschool De Ark,Gedeputeerdenlaan 47,/basisscholen/zwolle/5569/pc-basisschool-de-ark/,Prot. christelijk,225.0,Zwolle,Basic School
7197,Chr. Basisschool Het Mozaiek,Hortensiastraat 31,/basisscholen/zwolle/5570/chr-basisschool-het-...,Prot. christelijk,170.0,Zwolle,Basic School


In [67]:
primary_schools.to_csv('primary_schools-temporary.csv', index=False)

In [80]:
middle_schools

,name,address,link,details,n_students,neighborhood,type
0,Yuverta vmbo Aalsmeer,Jac. P. Thijsselaan 18,/middelbare-scholen/aalsmeer/1713/yuverta-vmbo...,Overige Voortgezet,535.0,Aalsmeer,Middle School
1,Yuverta mavo Aalsmeer,1e J.C. Mensinglaan 40,/middelbare-scholen/aalsmeer/1717/yuverta-mavo...,Overige Voortgezet,491.0,Aalsmeer,Middle School
2,Chr Coll Schaersvoorde,Slingelaan 28,/middelbare-scholen/aalten/3289/chr-coll-schae...,Prot. Christelijk,1019.0,Aalten,Middle School
3,Chr Coll Schaersvoorde,Landbouwstraat 1,/middelbare-scholen/aalten/23137/chr-coll-scha...,Prot. Christelijk,382.0,Aalten,Middle School
4,Montessori College Aerdenhout,Boekenroodeweg 5,/middelbare-scholen/aerdenhout/2536/montessori...,Samenwerking Voortgezet,427.0,Aerdenhout,Middle School
...,...,...,...,...,...,...,...
1570,de Ambelt,Dokter Hengeveldweg 9,/middelbare-scholen/zwolle/26613/de-ambelt/,Algemeen Bijzonder,174.0,Zwolle,Middle School
1571,De Ambelt,Herfterlaan 39 F,/middelbare-scholen/zwolle/26615/de-ambelt/,Algemeen Bijzonder,209.0,Zwolle,Middle School
1572,Van der Capellen Sg,Lassuslaan 230,/middelbare-scholen/zwolle/22431/van-der-capel...,Openbaar Voortgezet,1244.0,Zwolle,Middle School
1573,De Twijn/Dr Itardschool,Dokter Hengeveldweg 2,/middelbare-scholen/zwolle/22935/de-twijn-dr-i...,Openbaar Voortgezet,385.0,Zwolle,Middle School


In [112]:
def target_scrape_primary(dataframe):
    targets = []
    errors = []
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    try:
        for url in dataframe['link']:
            try:
                driver.get('https://scholenopdekaart.nl' + url + 'resultaten/')
                html = driver.page_source
                soup = BeautifulSoup(html)
                soup.find_all(attrs={'class': re.compile(r'highcharts-series highcharts-series-0 highcharts-bar-series highcharts-color-0 highcharts-tracker')})

                elements = soup.findAll('g', class_='highcharts-label')
                target_results = elements[4].text.replace('%', '').replace(',', '.')
                other_results = elements[5].text.replace('%', '').replace(',', '.')

                target_dict = {'URL': url, 'School Target': float(target_results), 'Other_schools_target': float(other_results)}
                targets.append(target_dict)

            except Exception as e:
                error_message = f"Error occurred for URL {url}: {str(e)}"
                print(error_message)
                errors.append({'URL': url, 'Error': error_message})

        target_df = pd.DataFrame(targets)
        errors_target_df = pd.DataFrame(errors)
        return target_df, errors_target_df

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame(columns=['URL', 'School_Target', 'Other_schools_target']), pd.DataFrame(columns=['URL', 'Error'])

In [113]:
def fundamental_scrape_primary(dataframe):
    fundamentals = []
    errors = []
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    try:
        for url in dataframe['link']:
            try:
                driver.get('https://scholenopdekaart.nl' + url + 'resultaten/')
                html = driver.page_source
                soup = BeautifulSoup(html)
                soup.find_all(attrs={'class': re.compile(r'highcharts-series highcharts-series-0 highcharts-bar-series highcharts-color-0 highcharts-tracker')})

                elements = soup.findAll('g', class_='highcharts-label')
                fundamental = elements[0].text.replace('%', '').replace(',', '.')
                other_results = elements[1].text.replace('%', '').replace(',', '.')

                fundamental_dict = {'URL': url, 'School fundamental': float(fundamental_results), 'Other_schools_fundamental': float(other_results)}
                fundamentals.append(fundamental_dict)

            except Exception as e:
                error_message = f"Error occurred for URL {url}: {str(e)}"
                print(error_message)
                errors.append({'URL': url, 'Error': error_message})

        fundamental_df = pd.DataFrame(fundamentals)
        errors_fundamental_df = pd.DataFrame(errors)
        return fundamental_df, errors_fundamental_df

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame(columns=['URL', 'School_fundamental', 'Other_schools_fundamental']), pd.DataFrame(columns=['URL', 'Error'])

In [114]:
def percentage_passing_students_middle(dataframe):
    passing_students = []
    errors = []
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    try:
        for url in dataframe['link']:
            try:
                driver.get('https://scholenopdekaart.nl' + url + 'resultaten/')
                html = driver.page_source
                soup = BeautifulSoup(html)
                soup.find_all(attrs={'class': re.compile(r'highcharts-markers highcharts-series-0 highcharts-area-series highcharts-color-0 highcharts-tracker')})

                elements = soup.findAll('g', class_='highcharts-label')
                students_who_passed = elements[16].text.replace('%','')

                students_who_passed_dict = {'URL': url, 'Students_who_passed': float(students_that_passed)}
                passing_students.append(students_who_passed_dict)

            except Exception as e:
                error_message = f"Error occurred for URL {url}: {str(e)}"
                print(error_message)
                errors.append({'URL': url, 'Error': error_message})

        passing_students_df = pd.DataFrame(passing_students)
        errors_students_df = pd.DataFrame(errors)
        return passing_students_df, errors_students_df

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame(columns=['URL', 'Percentage_passing_student']), pd.DataFrame(columns=['URL', 'Error'])
    

In [115]:
def avg_score_exam_middle(dataframe):
    avg_scores = []
    errors = []
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    try:
        for url in dataframe['link']:
            try:
                driver.get('https://scholenopdekaart.nl' + url + 'resultaten/')
                html = driver.page_source
                soup = BeautifulSoup(html)
                soup.find_all(attrs={'class': re.compile(r'highcharts-markers highcharts-series-0 highcharts-area-series highcharts-color-0 highcharts-tracker')})

                elements = soup.findAll('g', class_='highcharts-label')
                avg_score = elements[3].text

                avg_score_dict = {'URL': url, 'Students_who_passed': float(avg_score)}
                avg_scores.append(avg_score_dict)

            except Exception as e:
                error_message = f"Error occurred for URL {url}: {str(e)}"
                print(error_message)
                errors.append({'URL': url, 'Error': error_message})

        avg_scores_df = pd.DataFrame(avg_scores)
        errors_scores_df = pd.DataFrame(errors)
        return avg_scores_df, errors_scores_df

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return pd.DataFrame(columns=['URL', 'Avg_score_exam']), pd.DataFrame(columns=['URL', 'Error'])
    

In [116]:
targets, errors = target_scrape_primary(primary_schools)

Error occurred for URL /basisscholen/alkmaar/23774/burgemeester-de-wildeschool-alkmaar/: list index out of range
Error occurred for URL /basisscholen/almelo/23317/de-rietpluim/: list index out of range
Error occurred for URL /basisscholen/almere/23773/alexander-roozendaalschool-almere/: list index out of range
Error occurred for URL /basisscholen/almere/4998/nautilus-college-radioweg/: list index out of range
Error occurred for URL /basisscholen/almere/26221/nautilus-so/: list index out of range
Error occurred for URL /basisscholen/amerongen/24084/wereldkidz-sterrenbos/: list index out of range
Error occurred for URL /basisscholen/amersfoort/9907/auris-professor-groenschool/: list index out of range
Error occurred for URL /basisscholen/amstelveen/26506/wereldschool-amstelveen-nieuwkomersonderwijs/: list index out of range
Error occurred for URL /basisscholen/amstelveen/26507/amstelland-international-school/: list index out of range
Error occurred for URL /basisscholen/amsterdam/23894/p

In [117]:
targets

,URL,School Target,Other_schools_target
0,/basisscholen/2e-exloërmond/25159/kindcentrum-...,65.3,55.8
1,/basisscholen/2e-exloërmond/7595/openbare-basi...,51.4,48.9
2,/basisscholen/aadorp/11785/basisschool-de-twee...,64.7,58.4
3,/basisscholen/aagtekerke/25963/jhr-willem-vers...,72.6,55.8
4,/basisscholen/aalden/9661/christelijke-basissc...,73.3,61.1
...,...,...,...
3869,/basisscholen/roermond/10940/school-voor-speci...,9.3,39.3
3870,/basisscholen/roggel/4771/basisschool-de-zjwiek/,59.4,61.1
3871,/basisscholen/rolde/7482/basisschool-jan-thies/,71.0,63.6
3872,/basisscholen/roodeschool/4682/sws-de-dobbe/,47.2,55.6


In [121]:
errors.value_counts()

URL                                                               Error                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 